In [149]:
import osmapi as osm
from shapely.geometry import Polygon, Point
from random import uniform
from sklearn.neighbors import DistanceMetric
from sklearn.linear_model import LinearRegression
import folium

# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Тарелкин Александр Алексеевич

Группа: MADE-ML-11

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [15]:
AF_RELATION_ID = 303427

api = osm.OsmApi()

relation = api.RelationGet(AF_RELATION_ID)

In [63]:
relation

{'id': 303427,
 'visible': True,
 'version': 162,
 'changeset': 92480698,
 'timestamp': datetime.datetime(2020, 10, 14, 15, 11, 50),
 'user': 'Garmin-User',
 'uid': 177389,
 'tag': {'admin_level': '2',
  'alt_name:af': 'Afganistan',
  'alt_name:eo': 'Afganujo;Afganistano',
  'alt_name:gd': 'Afghanastàn',
  'alt_name:io': 'Afgania',
  'alt_name:it': 'Afganistan',
  'alt_name:la': 'Afghanistania',
  'alt_name:li': 'Afganistan',
  'alt_name:mn': 'Афган улс',
  'alt_name:mzn': 'افغانستون',
  'alt_name:sh': 'Avganistan',
  'alt_name:so': 'Afqaanistan',
  'alt_name:vec': 'Afghanistan',
  'alt_name:vi': 'Áp-ga-nít-xtăng',
  'alt_name:war': 'Afghanistan',
  'boundary': 'administrative',
  'default_language': 'ps',
  'flag': 'http://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg',
  'ISO3166-1': 'AF',
  'ISO3166-1:alpha2': 'AF',
  'ISO3166-1:alpha3': 'AFG',
  'ISO3166-1:numeric': '004',
  'name': 'افغانستان',
  'name:ace': 'Afghanistan',
  'name:ady': 'Афганистан',
  'name:

In [22]:
ways_ids = [x['ref'] for x in relation['member'] if x['type'] == 'way'] # получаем id путей границы

ways = api.WaysGet(ways_ids) # получаем сами пути

In [93]:
nodes_ids = [y for x in ways.values() for y in x['nd']] # получаем id точек границы

nodes_ids = [x for i,x in enumerate(nodes_ids) if i % 5 == 0] # берем каждую пятую точку границы, для быстродействия 

In [94]:
coords = {}
for i, x in enumerate(nodes_ids):
    if i % 25 == 0:
        print(i)
    node = api.NodeGet(x)
    coords[i] = (node['lat'], node['lon'])
    
af_poly = Polygon(c).convex_hull # полигон Афганистана

0
25
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
50
75
100
125
150
175
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1100
1125
1150
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
2050
('Connection aborted.', RemoteDisconnected('Remote end closed connection

In [227]:
# наивный метод

up_boundary_lat = 90
down_boundary_lat = -90
up_boundary_lon = 180
down_boundary_lon = 0

m = folium.Map(
    location=[af_poly.centroid.x, af_poly.centroid.y],
    zoom_start=1,
)

p = Point((uniform(down_boundary_lat, up_boundary_lat), uniform(down_boundary_lon, up_boundary_lon)))  
i = 0
while not af_poly.contains(p):
    p = Point((uniform(down_boundary_lat, up_boundary_lat), uniform(down_boundary_lon, up_boundary_lon)))
    i += 1
    folium.Marker(
        location=[p.x, p.y],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)
    
print(i, f'iteration')

45 iteration


In [222]:
m

In [139]:
def calc_haversine_dist_from_point_to_Af(lat, lon):
    """ возвращает расстояние от точки до центра Афганистана в км """
    
    af_center_coords = list(map(lambda x: x*3.1415925/180, [af_poly.centroid.x, af_poly.centroid.y]))
    point_coords = list(map(lambda x: x*3.1415925/180, [lat, lon]))
    
    haversine = DistanceMetric.get_metric('haversine')
    
    return 6371 * haversine.pairwise([af_center_coords, point_coords])[0][1]

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here